In [1]:
import pandas as pd
import numpy as np
from pyjet import cluster,DTYPE_PTEPM

# Pre processing

In [2]:
path = path = '/anomalyvol/data/events_LHCO2020_backgroundMC_Pythia.h5'

In [3]:
df = pd.read_hdf(path,stop=1000) # just read first 1000 events
df

,0,1,2,3,4,5,6,7,8,9,...,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099
0,3.587869,-2.323472,-2.597121,1.497173,-2.480994,-2.269457,0.848844,-2.465643,-2.096595,0.961511,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.921213,-0.745233,1.018857,0.689363,-0.642245,3.050711,1.999174,-0.343135,-0.322586,1.580572,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.580352,-2.412026,1.680236,0.429869,-0.778697,-1.453413,0.856914,-2.243512,0.217628,0.407344,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.579134,-0.243543,-2.561824,0.312690,-0.283086,-0.281626,0.775053,-2.062494,-1.598718,0.868891,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.644219,-2.457281,-2.670996,0.186128,-1.757650,2.719159,0.346987,-2.318233,-0.155036,0.501437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.493004,-0.886976,-0.391002,0.534181,-2.081904,2.548825,0.458036,-1.230976,2.204294,0.639672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.404833,-0.510012,2.969482,0.300500,-2.248194,1.012964,0.316375,-1.815956,1.011110,0.604675,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.407082,-2.195407,2.632497,0.413497,-1.212703,2.704103,0.578276,-0.291654,1.065150,0.508287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.573155,-0.755578,0.347725,0.751455,-1.929361,0.759764,0.798534,-0.532149,0.979545,0.941284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
all_events = df.values

In [5]:
rows = all_events.shape[0]
cols = all_events.shape[1]
data = np.zeros((rows, cols // 3, 4))

In [6]:
for i in range(rows):
    pseudojets_input = np.zeros(len([x for x in all_events[i][::3] if x > 0]), dtype=DTYPE_PTEPM)
    for j in range(cols // 3):
        if (all_events[i][j*3]>0):
            pseudojets_input[j]['pT'] = all_events[i][j*3]
            pseudojets_input[j]['eta'] = all_events[i][j*3+1]
            pseudojets_input[j]['phi'] = all_events[i][j*3+2]
        pass
    sequence = cluster(pseudojets_input, R=1.0, p=-1)
    jets = sequence.inclusive_jets()
    for k in range(len(jets)):
        data[i][k][0] = jets[k].pt
        data[i][k][1] = jets[k].eta
        data[i][k][2] = jets[k].phi
        data[i][k][3] = jets[k].mass

In [7]:
data.shape

(1000, 700, 4)

# VAE Model

In [8]:
import keras
from keras.layers import Lambda, Input, Dense, Flatten, Reshape
from keras.models import Model, Sequential
from keras import metrics
from keras import backend as K

Using TensorFlow backend.


In [9]:
def sampling(args):
    """
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [10]:
inter_dim = 32
final_dim = 8
latent_dim = 2
input_shape = (cols // 3, 4)

In [11]:
# encoder
x = Input(shape=input_shape)
x_flat = Flatten()(x)
h1 = Dense(inter_dim, activation='relu')(x_flat)
h2 = Dense(final_dim, activation='relu')(h1)
z_mean = Dense(latent_dim)(h2)
z_log_sigma = Dense(latent_dim)(h2)

In [12]:
# random sampling
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

encoder = Model(inputs = x, outputs = z)

In [13]:
# decoder
decoder_h2 = Dense(final_dim, activation='relu')
decoder_h1 = Dense(inter_dim, activation='relu')
decoder_mean = Dense(np.prod(input_shape), activation='sigmoid')

h2_decoded = decoder_h2(z)
h1_decoded = decoder_h1(h2_decoded)
x_decoded_mean = decoder_mean(h1_decoded)
x_decoded = Reshape(input_shape)(x_decoded_mean)

In [14]:
vae = Model(inputs = x, outputs = x_decoded, name = 'vae')

In [15]:
def vae_loss(x, y):
    xent_loss = metrics.binary_crossentropy(K.flatten(x), K.flatten(y))
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

vae.compile(optimizer='adam', loss=vae_loss)

In [16]:
vae.summary()

Model: "vae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 700, 4)       0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2800)         0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           89632       flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 8)            264         dense_1[0][0]                    
________________________________________________________________________________________________

# Train

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train = data[0:int(len(data) * 0.8)]
x_val = data[int(len(data) * 0.8):]
batch_size = 100
epochs = 150

In [19]:
x_train.shape

(800, 700, 4)

In [20]:
hist = vae.fit(x_train, x_train,
               shuffle=True,
               epochs=epochs,
               batch_size=batch_size,
               validation_data=(x_val, x_val))

Train on 800 samples, validate on 200 samples
Epoch 1/150
800/800 [==============================] - 1s 1ms/step - loss: 38634135.4211 - val_loss: 249.6192
Epoch 2/150
800/800 [==============================] - 0s 119us/step - loss: 291.6356 - val_loss: 402.7793
Epoch 3/150
800/800 [==============================] - 0s 128us/step - loss: 389.7881 - val_loss: 1724.2498
Epoch 4/150
800/800 [==============================] - 0s 130us/step - loss: 422.2971 - val_loss: 554.9063
Epoch 5/150
800/800 [==============================] - 0s 111us/step - loss: 223.8300 - val_loss: 215.8408
Epoch 6/150
800/800 [==============================] - 0s 117us/step - loss: 182.0653 - val_loss: 173.9011
Epoch 7/150
800/800 [==============================] - 0s 122us/step - loss: 159.9630 - val_loss: 153.5538
Epoch 8/150
800/800 [==============================] - 0s 120us/step - loss: 145.0281 - val_loss: 139.6160
Epoch 9/150
800/800 [==============================] - 0s 119us/step - loss: 132.0679 - val_lo

800/800 [==============================] - 0s 112us/step - loss: -10.9425 - val_loss: -10.7185
Epoch 78/150
800/800 [==============================] - 0s 113us/step - loss: -11.0467 - val_loss: -10.8339
Epoch 79/150
800/800 [==============================] - 0s 108us/step - loss: -11.1273 - val_loss: -10.9416
Epoch 80/150
800/800 [==============================] - 0s 117us/step - loss: -11.1489 - val_loss: -11.0091
Epoch 81/150
800/800 [==============================] - 0s 114us/step - loss: -11.2414 - val_loss: -11.1079
Epoch 82/150
800/800 [==============================] - 0s 118us/step - loss: -11.3062 - val_loss: -11.1705
Epoch 83/150
800/800 [==============================] - 0s 117us/step - loss: -11.3329 - val_loss: -11.2069
Epoch 84/150
800/800 [==============================] - 0s 112us/step - loss: -11.3766 - val_loss: -11.2510
Epoch 85/150
800/800 [==============================] - 0s 123us/step - loss: -11.4137 - val_loss: -11.3143
Epoch 86/150
800/800 [===================